In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

/home/joaquin/Documents/GitHub/skforecast


In [2]:
from skforecast.drift_detection import RangeDriftDetector
from pprint import pprint
from copy import deepcopy

In [3]:
y = pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D'), name="y")
series_df = pd.DataFrame(
    {
        'series_1': pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D')),
        'series_2': pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D')),
    }
)
series_df_long = series_df.stack().to_frame().rename(columns={0: 'y'}).swaplevel(0,1).sort_index()
exog = pd.Series(np.random.rand(5000), index=y.index, name="temp")
exog_df = pd.DataFrame(index=y.index)
for i in range(5):
    exog_df[f"feature_{i}"] = np.random.rand(5000)
for i in range(5):
    exog_df[f"cat_feature_{i}"] = np.random.choice(["A", "B", "C"], 5000)


series_dict = series_df.to_dict(orient='series')
exog_dict = {'series_1': exog_df, 'series_2': exog_df}
exog_df_long = [
    exog_df.copy().assign(series_id='series_1'),
    exog_df.copy().assign(series_id='series_2')
]
exog_df_long = pd.concat(exog_df_long).set_index('series_id', append=True).reorder_levels(['series_id', None]).sort_index()

# FIT

In [4]:
range_detector = RangeDriftDetector()
range_detector.fit(y)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'y': (np.float64(5.4714584017290235e-05), np.float64(0.9995938713662653))}
['y']


In [5]:
range_detector = RangeDriftDetector()
range_detector.fit(series_df)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(1.2887278784790013e-05),
              np.float64(0.9998196411799491)),
 'series_2': (np.float64(6.68112581787561e-05), np.float64(0.9998622511876596))}
['series_1', 'series_2']


In [6]:
range_detector = RangeDriftDetector()
range_detector.fit(series_dict)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(1.2887278784790013e-05),
              np.float64(0.9998196411799491)),
 'series_2': (np.float64(6.68112581787561e-05), np.float64(0.9998622511876596))}
['series_1', 'series_2']


In [7]:
range_detector = RangeDriftDetector()
range_detector.fit(series_df_long)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(1.2887278784790013e-05),
              np.float64(0.9998196411799491)),
 'series_2': (np.float64(6.68112581787561e-05), np.float64(0.9998622511876596))}
['series_1', 'series_2']


In [8]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'temp': (np.float64(0.0004021591381272449), np.float64(0.9997234955020412))}
['temp']


In [9]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_df)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'cat_feature_0': {'C', 'A', 'B'},
 'cat_feature_1': {'B', 'A', 'C'},
 'cat_feature_2': {'C', 'A', 'B'},
 'cat_feature_3': {'B', 'A', 'C'},
 'cat_feature_4': {'B', 'A', 'C'},
 'feature_0': (np.float64(6.823815426026947e-05),
               np.float64(0.9998352346016657)),
 'feature_1': (np.float64(8.12728956698372e-05),
               np.float64(0.9996514966828843)),
 'feature_2': (np.float64(8.577611281046504e-05),
               np.float64(0.9999702670064071)),
 'feature_3': (np.float64(0.00022060656555045188),
               np.float64(0.9997172184503613)),
 'feature_4': (np.float64(0.0005154670114558835),
               np.float64(0.9998506930150942))}
['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'cat_feature_0', 'cat_feature_1', 'cat_feature_2', 'cat_feature_3', 'cat_feature_4']


In [10]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_dict)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'series_1': {'cat_feature_0': {'C', 'A', 'B'},
              'cat_feature_1': {'B', 'A', 'C'},
              'cat_feature_2': {'C', 'A', 'B'},
              'cat_feature_3': {'B', 'A', 'C'},
              'cat_feature_4': {'B', 'A', 'C'},
              'feature_0': (np.float64(6.823815426026947e-05),
                            np.float64(0.9998352346016657)),
              'feature_1': (np.float64(8.12728956698372e-05),
                            np.float64(0.9996514966828843)),
              'feature_2': (np.float64(8.577611281046504e-05),
                            np.float64(0.9999702670064071)),
              'feature_3': (np.float64(0.00022060656555045188),
                            np.float64(0.9997172184503613)),
              'feature_4': (np.float64(0.0005154670114558835),
                            np.float64(0.9998506930150942))},
 'series_2': {'cat_feature_0': {'C', 'A', 'B'},
              'cat_feature_1': {'B', 'A', 'C'},
              'cat_feature_2': {'C', 'A', '

In [11]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_df_long)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'series_1': {'cat_feature_0': {'C', 'A', 'B'},
              'cat_feature_1': {'B', 'A', 'C'},
              'cat_feature_2': {'C', 'A', 'B'},
              'cat_feature_3': {'B', 'A', 'C'},
              'cat_feature_4': {'B', 'A', 'C'},
              'feature_0': (np.float64(6.823815426026947e-05),
                            np.float64(0.9998352346016657)),
              'feature_1': (np.float64(8.12728956698372e-05),
                            np.float64(0.9996514966828843)),
              'feature_2': (np.float64(8.577611281046504e-05),
                            np.float64(0.9999702670064071)),
              'feature_3': (np.float64(0.00022060656555045188),
                            np.float64(0.9997172184503613)),
              'feature_4': (np.float64(0.0005154670114558835),
                            np.float64(0.9998506930150942))},
 'series_2': {'cat_feature_0': {'C', 'A', 'B'},
              'cat_feature_1': {'B', 'A', 'C'},
              'cat_feature_2': {'C', 'A', '

# Predict

In [12]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog)
pprint(range_detector.series_values_range_)
pprint(range_detector.exog_values_range_)
range_detector

{'y': (np.float64(5.4714584017290235e-05), np.float64(0.9995938713662653))}
{'temp': (np.float64(0.0004021591381272449), np.float64(0.9997234955020412))}


================== 
RangeDriftDetector 
================== 
Fitted series          = ['y'] 
Series value ranges    = {'y': (np.float64(5.4714584017290235e-05), np.float64(0.9995938713662653))} 
Fitted exogenous       = ['temp'] 
Exogenous value ranges = {'temp': (np.float64(0.0004021591381272449), np.float64(0.9997234955020412))}

In [13]:
last_window_valid = y.tail(100).copy()
last_window_invalid = y.tail(100).copy()
last_window_invalid.iloc[5] = 10_000    
#range_detector.predict(last_window=last_window_valid, verbose=True, suppress_warnings=True)
range_detector.predict(last_window=last_window_invalid, verbose=False, suppress_warnings=False)
range_detector


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'y' has values outside the range seen during training [0.00005, 0.99959]. This may   │
│ affect the accuracy of the predictions.                                              │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :254                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

================== 
RangeDriftDetector 
================== 
Fitted series          = ['y'] 
Series value ranges    = {'y': (np.float64(5.4714584017290235e-05), np.float64(0.9995938713662653))} 
Fitted exogenous       = ['temp'] 
Exogenous value ranges = {'temp': (np.float64(0.0004021591381272449), np.float64(0.9997234955020412))}

In [14]:
range_detector.fit(series_df)
last_window_valid_df = series_df.tail(100).copy()
last_window_invalid_df = series_df.tail(100).copy()
last_window_invalid_df.iloc[1, [0, 1]] = 10_000
range_detector.predict(last_window=last_window_valid_df)
range_detector.predict(last_window=last_window_invalid_df)
range_detector

╭───────────────── Out-of-range summary ─────────────────╮
│ Series:                                                │
│ No series with out-of-range values found.              │
│                                                        │
│ Exogenous Variables:                                   │
│ No exogenous variables with out-of-range values found. │
│                                                        │
│                                                        │
╰────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1' has values outside the range seen during training [0.00001, 0.99982].     │
│ This may affect the accuracy of the predictions.                                     │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :254                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2' has values outside the range seen during training [0.00007, 0.99986].     │
│ This may affect the accuracy of the predictions.                                     │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :254                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── Out-of-range summary ────────────────────────╮
│ Series:                                                              │
│ 'series_1' has values outside the observed range [0.00001, 0.99982]. │
│ 'series_2' has values outside the observed range [0.00007, 0.99986]. │
│                                                                      │
│ Exogenous Variables:                                                 │
│ No exogenous variables with out-of-range values found.               │
│                                                                      │
│                                                                      │
╰──────────────────────────────────────────────────────────────────────╯

================== 
RangeDriftDetector 
================== 
Fitted series          = ['series_1', 'series_2'] 
Series value ranges    = {'series_1': (np.float64(1.2887278784790013e-05), np.float64(0.9998196411799491)), 'series_2': (np.float64(6.68112581787561e-05), np.float64(0.9998622511876596))} 
Fitted exogenous       = None 
Exogenous value ranges = None

In [15]:
range_detector.fit(series_dict)
last_window_valid_dict = {key: val.tail(100).copy() for key, val in series_dict.items()}
last_window_invalid_dict = {key: val.tail(100).copy() for key, val in series_dict.items()}
for key in last_window_invalid_dict.keys():
    last_window_invalid_dict[key].iloc[1] = 10_000
range_detector.predict(last_window=last_window_valid_dict)
range_detector.predict(last_window=last_window_invalid_dict)
range_detector

╭───────────────── Out-of-range summary ─────────────────╮
│ Series:                                                │
│ No series with out-of-range values found.              │
│                                                        │
│ Exogenous Variables:                                   │
│ No exogenous variables with out-of-range values found. │
│                                                        │
│                                                        │
╰────────────────────────────────────────────────────────╯

╭──────────────────────── Out-of-range summary ────────────────────────╮
│ Series:                                                              │
│ 'series_1' has values outside the observed range [0.00001, 0.99982]. │
│ 'series_2' has values outside the observed range [0.00007, 0.99986]. │
│                                                                      │
│ Exogenous Variables:                                                 │
│ No exogenous variables with out-of-range values found.               │
│                                                                      │
│                                                                      │
╰──────────────────────────────────────────────────────────────────────╯

================== 
RangeDriftDetector 
================== 
Fitted series          = ['series_1', 'series_2'] 
Series value ranges    = {'series_1': (np.float64(1.2887278784790013e-05), np.float64(0.9998196411799491)), 'series_2': (np.float64(6.68112581787561e-05), np.float64(0.9998622511876596))} 
Fitted exogenous       = None 
Exogenous value ranges = None

In [16]:
range_detector.fit(series_df_long)
last_window_valid_df_long= series_df_long.groupby(level=0).tail(100).copy()
last_window_invalid_df_long = series_df_long.groupby(level=0).tail(100).copy()
for key in last_window_invalid_df_long.index.get_level_values(0).unique():
    last_window_invalid_df_long.loc[key, pd.Timestamp("2013-06-02")] = 10_000
range_detector.predict(last_window=last_window_valid_df_long)
range_detector.predict(last_window=last_window_invalid_df_long)

╭───────────────── Out-of-range summary ─────────────────╮
│ Series:                                                │
│ No series with out-of-range values found.              │
│                                                        │
│ Exogenous Variables:                                   │
│ No exogenous variables with out-of-range values found. │
│                                                        │
│                                                        │
╰────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1' has values outside the range seen during training [0.00001, 0.99982].     │
│ This may affect the accuracy of the predictions.                                     │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :254                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2' has values outside the range seen during training [0.00007, 0.99986].     │
│ This may affect the accuracy of the predictions.                                     │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :254                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── Out-of-range summary ────────────────────────╮
│ Series:                                                              │
│ 'series_1' has values outside the observed range [0.00001, 0.99982]. │
│ 'series_2' has values outside the observed range [0.00007, 0.99986]. │
│                                                                      │
│ Exogenous Variables:                                                 │
│ No exogenous variables with out-of-range values found.               │
│                                                                      │
│                                                                      │
╰──────────────────────────────────────────────────────────────────────╯

(True, ['series_1', 'series_2'], [])

In [17]:
range_detector.fit(series_df, exog=exog)
print(range_detector)
exog_valid = exog.tail(100).copy()
exog_invalid = exog.tail(100).copy()
exog_invalid.iloc[5] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

RangeDriftDetector 
Fitted series          = ['series_1', 'series_2'] 
Series value ranges    = {'series_1': (np.float64(1.2887278784790013e-05), np.float64(0.9998196411799491)), 'series_2': (np.float64(6.68112581787561e-05), np.float64(0.9998622511876596))} 
Fitted exogenous       = ['temp'] 
Exogenous value ranges = {'temp': (np.float64(0.0004021591381272449), np.float64(0.9997234955020412))} 



╭───────────────── Out-of-range summary ─────────────────╮
│ Series:                                                │
│ No series with out-of-range values found.              │
│                                                        │
│ Exogenous Variables:                                   │
│ No exogenous variables with out-of-range values found. │
│                                                        │
│                                                        │
╰────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'temp' has values outside the range seen during training [0.00040, 0.99972]. This    │
│ may affect the accuracy of the predictions.                                          │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :254                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────── Out-of-range summary ──────────────────────╮
│ Series:                                                          │
│ No series with out-of-range values found.                        │
│                                                                  │
│ Exogenous Variables:                                             │
│ 'temp' has values outside the observed range [0.00040, 0.99972]. │
│                                                                  │
│                                                                  │
╰──────────────────────────────────────────────────────────────────╯

(True, [], ['temp'])

In [18]:
range_detector = RangeDriftDetector()
range_detector.fit(series=y, exog=exog_df)
range_detector
exog_valid = exog_df.tail(100).copy()
exog_invalid = exog_df.tail(100).copy()
exog_invalid.iloc[5, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

╭───────────────── Out-of-range summary ─────────────────╮
│ Series:                                                │
│ No series with out-of-range values found.              │
│                                                        │
│ Exogenous Variables:                                   │
│ No exogenous variables with out-of-range values found. │
│                                                        │
│                                                        │
╰────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_0' has values outside the range seen during training [0.00007, 0.99984].    │
│ This may affect the accuracy of the predictions.                                     │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :254                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_1' has values outside the range seen during training [0.00008, 0.99965].    │
│ This may affect the accuracy of the predictions.                                     │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :254                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── Out-of-range summary ─────────────────────────╮
│ Series:                                                               │
│ No series with out-of-range values found.                             │
│                                                                       │
│ Exogenous Variables:                                                  │
│ 'feature_0' has values outside the observed range [0.00007, 0.99984]. │
│ 'feature_1' has values outside the observed range [0.00008, 0.99965]. │
│                                                                       │
│                                                                       │
╰───────────────────────────────────────────────────────────────────────╯

(True, [], ['feature_0', 'feature_1'])

In [19]:
range_detector = RangeDriftDetector()
range_detector.fit(series=y, exog=exog_df)
range_detector
exog_valid = exog_df.tail(100).copy()
exog_invalid = exog_df.tail(100).copy()
exog_invalid.iloc[5, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

╭───────────────── Out-of-range summary ─────────────────╮
│ Series:                                                │
│ No series with out-of-range values found.              │
│                                                        │
│ Exogenous Variables:                                   │
│ No exogenous variables with out-of-range values found. │
│                                                        │
│                                                        │
╰────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_0' has values outside the range seen during training [0.00007, 0.99984].    │
│ This may affect the accuracy of the predictions.                                     │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :254                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_1' has values outside the range seen during training [0.00008, 0.99965].    │
│ This may affect the accuracy of the predictions.                                     │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :254                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── Out-of-range summary ─────────────────────────╮
│ Series:                                                               │
│ No series with out-of-range values found.                             │
│                                                                       │
│ Exogenous Variables:                                                  │
│ 'feature_0' has values outside the observed range [0.00007, 0.99984]. │
│ 'feature_1' has values outside the observed range [0.00008, 0.99965]. │
│                                                                       │
│                                                                       │
╰───────────────────────────────────────────────────────────────────────╯

(True, [], ['feature_0', 'feature_1'])

In [20]:
range_detector = RangeDriftDetector()

range_detector.fit(series=series_dict, exog=exog_dict)
print(range_detector)
exog_valid = deepcopy(exog_dict)
exog_invalid = deepcopy(exog_dict)
for key in exog_invalid.keys():
    
    exog_invalid[key].iloc[1, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)
range_detector

RangeDriftDetector 
Fitted series          = ['series_1', 'series_2'] 
Series value ranges    = {'series_1': (np.float64(1.2887278784790013e-05), np.float64(0.9998196411799491)), 'series_2': (np.float64(6.68112581787561e-05), np.float64(0.9998622511876596))} 
Fitted exogenous       = ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'cat_feature_0', 'cat_feature_1', 'cat_feature_2', 'cat_feature_3', 'cat_feature_4'] 
Exogenous value ranges = {'series_1': {'feature_0': (np.float64(6.823815426026947e-05), np.float64(0.9998352346016657)), 'feature_1': (np.float64(8.12728956698372e-05), np.float64(0.9996514966828843)), 'feature_2': (np.float64(8.577611281046504e-05), np.float64(0.9999702670064071)), 'feature_3': (np.float64(0.00022060656555045188), np.float64(0.9997172184503613)), 'feature_4': (np.float64(0.0005154670114558835), np.float64(0.9998506930150942)), 'cat_feature_0': {'C', 'A', 'B'}, 'cat_feature_1': {'B', 'A', 'C'}, 'cat_feature_2': {'C', 'A', 'B'}, 'cat_feature

╭───────────────── Out-of-range summary ─────────────────╮
│ Series:                                                │
│ No series with out-of-range values found.              │
│                                                        │
│ Exogenous Variables:                                   │
│ No exogenous variables with out-of-range values found. │
│                                                        │
│                                                        │
╰────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1': 'feature_0' has values outside the range seen during training [0.00007,  │
│ 0.99984]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :254                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1': 'feature_1' has values outside the range seen during training [0.00008,  │
│ 0.99965]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :254                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': 'feature_0' has values outside the range seen during training [0.00007,  │
│ 0.99984]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :254                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': 'feature_1' has values outside the range seen during training [0.00008,  │
│ 0.99965]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :254                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────── Out-of-range summary ──────────────────────────╮
│ Series:                                                                 │
│ No series with out-of-range values found.                               │
│                                                                         │
│ Exogenous Variables:                                                    │
│ 'series_1': 'feature_0' has values outside the observed range [0.00007, │
│ 0.99984].                                                               │
│ 'series_1': 'feature_1' has values outside the observed range [0.00008, │
│ 0.99965].                                                               │
│ 'series_2': 'feature_0' has values outside the observed range [0.00007, │
│ 0.99984].                                                               │
│ 'series_2': 'feature_1' has values outside the observed range [0.00008, │
│ 0.99965].                                                               │
│                                                                         │
│                                                                         │
╰─────────────────────────────────────────────────────────────────────────╯

================== 
RangeDriftDetector 
================== 
Fitted series          = ['series_1', 'series_2'] 
Series value ranges    = {'series_1': (np.float64(1.2887278784790013e-05), np.float64(0.9998196411799491)), 'series_2': (np.float64(6.68112581787561e-05), np.float64(0.9998622511876596))} 
Fitted exogenous       = ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'cat_feature_0', 'cat_feature_1', 'cat_feature_2', 'cat_feature_3', 'cat_feature_4'] 
Exogenous value ranges = {'series_1': {'feature_0': (np.float64(6.823815426026947e-05), np.float64(0.9998352346016657)), 'feature_1': (np.float64(8.12728956698372e-05), np.float64(0.9996514966828843)), 'feature_2': (np.float64(8.577611281046504e-05), np.float64(0.9999702670064071)), 'feature_3': (np.float64(0.00022060656555045188), np.float64(0.9997172184503613)), 'feature_4': (np.float64(0.0005154670114558835), np.float64(0.9998506930150942)), 'cat_feature_0': {'C', 'A', 'B'}, 'cat_feature_1': {'B', 'A', 'C'}, 'cat_feature_2': {'C', 'A', 'B'}, 'cat_feature_3': {'B', 'A', 'C'}, 'cat_feature_4': {'B', 'A', 'C'}}, 'series_2': {'feature_0': (np.float64(6.823815426026947e-05), np.float64(0.9998352346016657)), 'feature_1': (np.float64(8.12728956698372e-05), np.float64(0.9996514966828843)), 'feature_2': (np.float64(8.577611281046504e-05), np.float64(0.9999702670064071)), 'feature_3': (np.float64(0.00022060656555045188), np.float64(0.9997172184503613)), 'feature_4': (np.float64(0.0005154670114558835), np.float64(0.9998506930150942)), 'cat_feature_0': {'C', 'A', 'B'}, 'cat_feature_1': {'B', 'A', 'C'}, 'cat_feature_2': {'C', 'A', 'B'}, 'cat_feature_3': {'B', 'A', 'C'}, 'cat_feature_4': {'B', 'A', 'C'}}}

In [21]:
series = pd.DataFrame({
    'series_1': [1, 2, 3, 4, 5],
    'series_2': [10, 20, 30, 40, 50]
})
exog = pd.DataFrame({
    'exog_1': [10, 20, 30, 40, 50],
    'exog_2': ['A', 'B', 'C', 'D', 'E']
})
detector = RangeDriftDetector()
detector.fit(series=series, exog=exog)

assert detector.is_fitted is True
assert detector.series_names_in_ == ['series_1', 'series_2']
assert detector.series_values_range_ == {
    'series_1': (1.0, 5.0),
    'series_2': (10.0, 50.0)
}
assert detector.exog_names_in_ == ['exog_1', 'exog_2']
assert detector.exog_values_range_ == {
    'exog_1': (10.0, 50.0),
    'exog_2': set(['A', 'B', 'C', 'D', 'E'])
}

In [22]:
pd.DataFrame({
        'series_1': [1, 2, 3, 4, 5],
        'series_2': [10, 20, 30, 40, 50]
    }).stack().to_frame().swaplevel(0,1).sort_index()   

0
series_1 0   1
         1   2
         2   3
         3   4
         4   5
series_2 0  10
         1  20
         2  30
         3  40
         4  50

In [23]:
index = pd.MultiIndex.from_tuples(
        [('series_1', 0), ('series_1', 1), ('series_1', 2),
         ('series_2', 0), ('series_2', 1), ('series_2', 2),
         ('series_3', 0), ('series_3', 1), ('series_3', 2)],
        names=['series', 'time']
    )
index

MultiIndex([('series_1', 0),
            ('series_1', 1),
            ('series_1', 2),
            ('series_2', 0),
            ('series_2', 1),
            ('series_2', 2),
            ('series_3', 0),
            ('series_3', 1),
            ('series_3', 2)],
           names=['series', 'time'])

In [24]:
import pytest
import pandas as pd
import numpy as np
import warnings
from skforecast.drift_detection import RangeDriftDetector

series = pd.Series([1, 2, 3], name='y')
detector = RangeDriftDetector()
detector.fit(y=series)

detector

================== 
RangeDriftDetector 
================== 
Fitted series          = ['y'] 
Series value ranges    = {'y': (np.int64(1), np.int64(3))} 
Fitted exogenous       = None 
Exogenous value ranges = None